In [1]:
import geopandas as gpd 
import rasterio  
import numpy as np  

import ee
import geemap
from datetime import datetime 

In [10]:
# Load the GeoJSON file 
gdf = gpd.read_file("train.geojson")

test_gdf = gpd.read_file("test.geojson")

# Load your GeoJSON file
# gdf = gpd.read_file("train.geojson")

# Save as Shapefile (this creates .shp, .shx, .dbf, .prj)
# gdf.to_file("train_shapefile", driver="ESRI Shapefile")

test_gdf.to_file("test_shapefile", driver="ESRI Shapefile")



In [3]:
minx, miny, maxx, maxy = gdf.total_bounds

print(f"Bounding Box Coordinates (EPSG:{gdf.crs.to_epsg()}):")
print(f"West (min longitude): {minx}")
print(f"South (min latitude): {miny}")
print(f"East (max longitude): {maxx}")
print(f"North (max latitude): {maxy}")

Bounding Box Coordinates (EPSG:4326):
West (min longitude): -7.56471
South (min latitude): 4.549254027561672
East (max longitude): -2.888562331031698
North (max latitude): 7.48906023319121


In [4]:
import requests

country = "Cote d'Ivoire"
url = f"https://nominatim.openstreetmap.org/search?q={country}&format=json&limit=1"
headers = {'User-Agent': 'Mozilla/5.0'}

response = requests.get(url, headers=headers)
data = response.json()

boundingbox = data[0]["boundingbox"]  
lat = data[0]["lat"]
lon = data[0]["lon"]

print("Bounding box:", boundingbox)
print("Center lat/lon:", lat, lon)


Bounding box: ['4.1621205', '10.7401970', '-8.6014675', '-2.4948836']
Center lat/lon: 7.9897371 -5.5679458


In [2]:

ee.Authenticate()
ee.Initialize()

bands = ['B3', 'B4', 'B8', 'B11', 'B12']  
indices = ['NDVI', 'NDWI', 'EVI']


cotedivore_bbox = ee.Geometry.Rectangle([-8.6, 4.16, -2.49, 10.74])
start_year = 2024
months = range(1,13)


In [4]:
def get_month_range(month):
    start = f'{start_year}-{month}-01'
    end = ee.Date(start).advance(1, 'month').format('YYYY-MM-dd')
    return ee.Date(start), ee.Date(end)

In [3]:
crops = ee.FeatureCollection("projects/ee-oderaisaack2/assets/train_shapefile")
crops_test = ee.FeatureCollection("projects/ee-oderaisaack2/assets/test_shapefile")

s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filterDate('2024-01-01', '2024-12-31') \
    .filterBounds(crops_test) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .select(['VV', 'VH'])

In [4]:
def monthly_composites(year, start_month, end_month):
    months = ee.List.sequence(start_month, end_month)
    
    def per_month(m):
        m = ee.Number(m)
        start = ee.Date.fromYMD(year, m, 1)
        end = start.advance(1, 'month')
        
        monthly = s1.filterDate(start, end).mean()
        
        rvi = monthly.expression(
            '(4 * VH) / (VV + VH)', {
                'VH': monthly.select('VH'),
                'VV': monthly.select('VV')
            }
        ).rename('RVI')
        
        monthly = monthly.addBands(rvi) \
                         .set('month', m) \
                         .set('system:time_start', start.millis())
        return monthly

    return ee.ImageCollection(months.map(per_month))

In [5]:
monthly_images = monthly_composites(2024, 1, 12)

In [6]:
def extract_features(img):
    month = img.get('month')
    return img.reduceRegions(
        collection=crops_test,
        reducer=ee.Reducer.mean(),
        scale=10
    ).map(lambda f: f.set('month', month))

In [7]:
features = monthly_images.map(extract_features).flatten()

In [8]:
# # crops = ee.FeatureCollection("projects/ee-oderaisaack2/assets/train_shapefile")
# features = stacked.reduceRegions(
#     collection=crops, 
#     reducer=ee.Reducer.mean(),
#     scale=10 
# ) 

# # Export to CSV


In [9]:
export = ee.batch.Export.table.toDrive(
    collection=features,
    description='S1_RVI_JJA_monthly',
    folder='EarthEngineExports',
    fileNamePrefix='Tests1_jan_dec',
    fileFormat='CSV'
)

export.start()

`Sentine 1 & 2 Harmonized`

In [ ]:
# def add_indices(img):
#     ndvi = img.normalizedDifference(['B8', 'B4']).rename('NDVI')  # NIR, RED
#     ndmi = img.normalizedDifference(['B8', 'B11']).rename('NDMI')  # NIR, SWIR1
#     savi = img.expression(
#         '((NIR - RED) / (NIR + RED + L)) * (1 + L)',
#         {
#             'NIR': img.select('B8'),
#             'RED': img.select('B4'),
#             'L': ee.Number(0.5)
#         }
#     ).rename('SAVI')
#     gndvi = img.normalizedDifference(['B8', 'B2']).rename('GNDVI')  # NIR, GREEN
#     msavi2 = img.expression(
#         '0.5 * (2 * NIR + 1 - sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - RED)))',
#         {
#             'NIR': img.select('B8'),
#             'RED': img.select('B4')
#         }
#     ).rename('MSAVI2')
#     nbr = img.normalizedDifference(['B8', 'B12']).rename('NBR')  # NIR, SWIR2
#     return img.addBands([ndvi, ndmi, savi, gndvi, msavi2, nbr])



In [10]:
crops = ee.FeatureCollection("projects/ee-oderaisaack2/assets/train_shapefile")

def mask_clouds_shadows(image):
    qa = image.select('QA60')
    cloud_mask = qa.bitwiseAnd(1 << 10).eq(0).And(qa.bitwiseAnd(1 << 11).eq(0))
    cloud_mask = cloud_mask.focal_max(2)

    # Use SWIR and NIR bands for shadows instead of B2 
    swir = image.select('B11')
    nir = image.select('B8')
    shadow_mask = swir.lt(500).And(nir.lt(1000))

    final_mask = cloud_mask.And(shadow_mask.Not())
    return image.updateMask(final_mask).multiply(0.0001).copyProperties(image, ['system:time_start'])


In [11]:
def add_indices(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    evi = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED + 1))',  # No B2
        {'NIR': image.select('B8'), 'RED': image.select('B4')}
    ).rename('EVI')
    return image.addBands([ndvi, evi])

In [12]:
def process_month(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')

    # Adjust cloud threshold dynamically
    rainy_months = [4, 5, 6, 7, 8, 9, 10, 11]
    cloud_thresh = 60 if month in rainy_months else 30

    collection = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterDate(start, end)
        .filterBounds(crops_test)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_thresh))
        .map(mask_clouds_shadows)
        .map(add_indices)
    )

    # Ensure collection is not empty
    size = collection.size()
    empty = size.eq(0)

    def compute_and_export():
        mean_image = collection.mean().select(bands + ['NDVI', 'EVI'])
        mean_image = mean_image.set('month', month)

        # Reduce over crop polygons
        features = mean_image.reduceRegions(
            collection=crops_test,
            reducer=ee.Reducer.mean(),
            scale=10
        )

        # Add month number to each feature
        features = features.map(lambda f: f.set('month', month))

        # Export as CSV
        export = ee.batch.Export.table.toDrive(
            collection=features,
            description=f'S2_Features_{year}_{month:02d}',
            folder='EarthEngineExports',
            fileNamePrefix=f'Test_s2_crop_features_{year}_{month:02d}',
            fileFormat='CSV'
        )
        export.start()

    # Use conditional to avoid errors
    return ee.Algorithms.If(empty, None, compute_and_export())


In [13]:
for m in range(1, 13):
    process_month(2024, m)

In [51]:
def monthly_composites(year):
    months = ee.List.sequence(1, 12)

    def create_month(m):
        m = ee.Number(m)
        start = ee.Date.fromYMD(year, m, 1)
        end = start.advance(1, 'month')
        monthly = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
                     .filterBounds(crops)
                     .filterDate(start, end)
                     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))  # Filtering Happens Here
                     .map(mask_clouds_shadows)
                     .map(add_indices)
                     )
        mean_img = monthly.mean().set('month', m)
        return mean_img

    monthly_collection = ee.ImageCollection(months.map(create_month))
    return monthly_collection

In [49]:
def print_band_names(image):
    band_names = image.bandNames()
    print(band_names)
    return image

In [55]:
# Get monthly stacked collection
year = 2024
monthly_s2 = monthly_composites(year)

# Extract stats per month per polygon
def extract_stats(img):
    month = img.get('month')
    reduced = img.reduceRegions(
        collection=crops,
        reducer=ee.Reducer.mean(),
        scale=100
    )
    return reduced.map(lambda f: f.set('month', month))

In [56]:
# features_s1s2 = monthly_s2.map(extract_stats).flatten()
features_s1s2 = monthly_s2.select(bands + ['NDVI', 'NDMI', 'SAVI', 'GNDVI', 'MSAVI2',
                                          'NBR']).map(extract_stats).flatten()  # Select bands here


# Export to Drive
export = ee.batch.Export.table.toDrive(
    collection=features_s1s2,
    description='Sentinel2_Features_AllMonths',
    folder='EarthEngineExports',
    fileNamePrefix='s2_crop_features_long',
    fileFormat='CSV'
)
export.start()

In [61]:
final_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 953 entries, 0 to 952
Data columns (total 29 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   ID            953 non-null    object  
 1   year          953 non-null    int32   
 2   crop          953 non-null    object  
 3   class         953 non-null    int32   
 4   geometry      953 non-null    geometry
 5   NDVI_2024_01  593 non-null    float32 
 6   EVI_2024_01   593 non-null    float32 
 7   NDVI_2024_02  595 non-null    float32 
 8   EVI_2024_02   595 non-null    float32 
 9   NDVI_2024_03  940 non-null    float32 
 10  EVI_2024_03   940 non-null    float32 
 11  NDVI_2024_04  318 non-null    float32 
 12  EVI_2024_04   318 non-null    float32 
 13  NDVI_2024_05  98 non-null     float32 
 14  EVI_2024_05   98 non-null     float32 
 15  NDVI_2024_06  176 non-null    float32 
 16  EVI_2024_06   176 non-null    float32 
 17  NDVI_2024_07  699 non-null    float32 
 18  EV

In [8]:
# gdf["centroid"] = gdf.geometry.centroid
test_gdf['centroid'] = test_gdf.geometry.centroid

# Open raster
with rasterio.open("cotedivoire_cop30.tif") as src:
    coords = [(x,y) for x, y in zip(test_gdf.centroid.x, test_gdf.centroid.y)]
    values = list(src.sample(coords))
    test_gdf["height_vals"] = [v[0] if v else None for v in values]

C:\Users\PC\AppData\Local\Temp\ipykernel_8556\1726586447.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  test_gdf['centroid'] = test_gdf.geometry.centroid
C:\Users\PC\AppData\Local\Temp\ipykernel_8556\1726586447.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  coords = [(x,y) for x, y in zip(test_gdf.centroid.x, test_gdf.centroid.y)]


In [6]:
print(gdf.crs.to_epsg())

with rasterio.open("cotedivoire_cop30.tif") as src:
    print(src.crs)  # Shows full CRS info
    print(src.crs.to_epsg())

4326
EPSG:4326
4326


In [9]:
test_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   ID           282 non-null    object  
 1   year         282 non-null    int32   
 2   geometry     282 non-null    geometry
 3   centroid     282 non-null    geometry
 4   height_vals  282 non-null    float32 
dtypes: float32(1), geometry(2), int32(1), object(1)
memory usage: 8.9+ KB


In [10]:
test_gdf.to_csv("test_elevation.csv")